In [1]:
import tensorflow as tf # 导入TensorFlow库 
import os # 导入OS库 

def echo_tensor(sess, tensor, prefix = ''):
    # 注意： print() 显示时会把元素之间的逗号去掉
    if( isinstance(tensor, tf.Tensor) ):
        print( '{0} tensor.shape = {1}, tensor = {2}\r\n'.format(prefix, tf.shape(tensor), sess.run(tensor)) )
        # print( '{0} tensor = {1}\r\n'.format(prefix, sess.run(tensor)) )
    else:
        print( '{0} not_tensor = {1}\r\n'.format(prefix, tensor) )


# 从csv文件读取数据，加载解析，创建批次读取张量多行数据
# 调用举例: read_csv(4, ['1.csv', '2.csv'], [[0], [0.], ['']])
# 
# 如果 csv 文件中数据为：
# 1,2,3,4
# 11,12,13,14
# 21,22,23,24
# 31,32,33,34
# 41,42,43,44
# 51,52,53,54
# 
# 那么输出会是：
# [[21, 51, 1, 61], [22, 52, 2, 62], [23, 53, 3, 63], [24, 54, 4, 64]]
# 
def read_csv(batch_size, file_name, record_defaults): #从csv文件读取数据，加载解析，创建批次读取张量多行数据
    file_path = list( map(lambda name: os.path.join(os.getcwd(), name), file_name) )
    file_queue = tf.train.string_input_producer(file_path) 
    
    reader = tf.TextLineReader(skip_header_lines = 1) # reader = tf.TFRecordReader()
    key, value = reader.read(file_queue) 

    decoded = tf.decode_csv(value, record_defaults = record_defaults) #字符串(文本行)转换到指定默认值张量列元组，为每列设置数据类型

    return tf.train.shuffle_batch(decoded, batch_size=batch_size, capacity=batch_size * 50, min_after_dequeue=batch_size) # 读取文件，加载张量batch_size行 


with tf.Session() as sess: # 会话对象启动数据流图，搭建流程 
    print( 'Session: start' )
    tf.initialize_all_variables().run() 

    batch_data = read_csv(4, ['iris.csv'], [[0.], [0.], [0.], [0.], ['']])

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess = sess, coord = coord)
    echo_tensor( sess, threads, 'threads_1' )
    
    
    try:
        # while not coord.should_stop():
            for i in range(3):
                echo_tensor( sess, batch_data, 'batch_data_' + str(i) )
            
            f1, f2, f3, f4, lable = batch_data
            echo_tensor(sess, f2, 'f2_1')
            echo_tensor(sess, f2, 'f2_2')
            echo_tensor(sess, lable, 'lable')
            
            # tf.pack 的数据类型必须一致
            is_class_1 = tf.to_float(tf.equal(lable, [b'Iris-setosa']))
            features = tf.pack([f1, f2, f3, f4, is_class_1])
            echo_tensor(sess, features, 'features')


    except tf.errors.OutOfRangeError:
        print( 'Done training -- epoch limit reached' )

    finally:
        # When done, ask the threads to stop.
        echo_tensor( sess, threads, 'threads_2' )
        coord.request_stop()        
        coord.join(threads)

    sess.close()    
    print( 'Session: close' )

Session: start
threads_1 not_tensor = [<Thread(Thread-4, started daemon 140709277452032)>, <Thread(Thread-5, started daemon 140709269059328)>, <Thread(Thread-6, started daemon 140709260666624)>, <Thread(Thread-7, started daemon 140709252273920)>]

batch_data_0 not_tensor = [<tf.Tensor 'shuffle_batch:0' shape=(4,) dtype=float32>, <tf.Tensor 'shuffle_batch:1' shape=(4,) dtype=float32>, <tf.Tensor 'shuffle_batch:2' shape=(4,) dtype=float32>, <tf.Tensor 'shuffle_batch:3' shape=(4,) dtype=float32>, <tf.Tensor 'shuffle_batch:4' shape=(4,) dtype=string>]

batch_data_1 not_tensor = [<tf.Tensor 'shuffle_batch:0' shape=(4,) dtype=float32>, <tf.Tensor 'shuffle_batch:1' shape=(4,) dtype=float32>, <tf.Tensor 'shuffle_batch:2' shape=(4,) dtype=float32>, <tf.Tensor 'shuffle_batch:3' shape=(4,) dtype=float32>, <tf.Tensor 'shuffle_batch:4' shape=(4,) dtype=string>]

batch_data_2 not_tensor = [<tf.Tensor 'shuffle_batch:0' shape=(4,) dtype=float32>, <tf.Tensor 'shuffle_batch:1' shape=(4,) dtype=float32>,